In [ ]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data_type = "train"
# month = "07"
# category = "청구입금정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type}/4.청구입금정보/'

In [ ]:
# 1. 데이터 불러오기
df = pd.read_parquet(f'{drive_folder}cleaned_청구정보.parquet')
df

# 2. 컬럼 분리
segment_cols = ['Segment_A', 'Segment_B', 'Segment_C', 'Segment_D', 'Segment_E']
X = df.drop(columns=['ID'] + segment_cols)
y = df[segment_cols].idxmax(axis=1)  # 'Segment_A' ~ 'Segment_E'

# ❗ 3. 문자열 라벨 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # → 0~4 숫자

# 4. train/test 분리
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# 5. 모델 학습
model = XGBClassifier(
    objective="multi:softmax",
    num_class=5,
    eval_metric="mlogloss",
    n_estimators=300,
    tree_method='hist',
    device='cuda',
    use_label_encoder=False,
    random_state=42,
)
model.fit(X_train, y_train)

In [ ]:
# 6. 중요도 추출
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("🎯 XGBoost Feature Importance - TOP 8")
print(importances.head(8))

In [ ]:
# 1. 상위 8개 feature만 추출
top_8_features = importances.head(8).index.tolist()

In [ ]:
# 2. X 기준과 행 일치 보장 + ID, 기준년월 포함해서 최종 컬럼 구성
df_selected = df.loc[X.index, ['ID', '기준년월'] + top_8_features]

In [ ]:
# 4. 저장
df_selected.to_parquet(f'{drive_folder}xgb_top8_청구정보.parquet', index=False)

In [ ]:
# 데이터 불러오기
df100 = pd.read_parquet(f'{drive_folder}xgb_top8_청구정보.parquet')
df100

In [ ]:
# 데이터 불러오기
df200 = pd.read_parquet('../data/open/train/xgb_top8_All.parquet')
df200

In [ ]:
df200.info()